In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##### Imports

In [2]:
import sys
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# import random
from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
from nltk import download
# from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict

import matplotlib.pyplot as plt

from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

# print("Downloading stop-words..")
download('stopwords') # nltk

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#### Code

In [0]:
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    base_path = "/content/drive/My Drive/ML_data/homework_4_kaggle/"
else:
    base_path = ""

TRAIN_DATA_PATH = base_path + "data/data_train.pkl"
TEST_DATA_PATH = base_path + "data/data_test.pkl"


train_data = pd.read_pickle(TRAIN_DATA_PATH)
test_data  = pd.read_pickle(TEST_DATA_PATH)

X      = np.array(train_data[0])
y      = np.array(train_data[1])
X_test = np.array(test_data)

##### Helper Functions

In [0]:
# create csv in desired submission format
def create_and_save_submission(predictions, file_name="submission.csv"):
    ids = [i for i in range(len(predictions))]
    sub_df = pd.DataFrame(data=list(zip(ids, predictions)), columns=["Id","Category"])
    sub_df.to_csv(file_name, index=False)
    

# evauate model score 
def evaluate_model(Model, X, y, kfold=True):
    accuracy = []

    if kfold:
        print("Evaluating model performance using k-fold...")
        
        kfold = KFold(
            n_splits=5,
            shuffle=True,
            random_state=42
        )
        

        for train_index, test_index in kfold.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            Model.fit(X_train, y_train)
            accuracy.append(Model.score(X_test, y_test))
    
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        Model.fit(X_train, y_train)
        accuracy.append(Model.score(X_test, y_test))
    
    return np.mean(accuracy)

##### Code

In [0]:
# classes = set(y)

# encode y
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(y)
vector_Y = label_encoder.transform(y)

In [6]:
# encode X
# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer(stop_words='english',  sublinear_tf=True, ngram_range=(1,1), strip_accents='unicode')
vectorizer.fit(X)
vector_X = vectorizer.transform(X)
vector_X_test = vectorizer.transform(X_test)

vector_X_test.shape, vector_X.shape

((30000, 73761), (70000, 73761))

In [0]:
mnb_clf = MultinomialNB(alpha=0.25)
brnb_clf = BernoulliNB(alpha=0.25)
random_clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
decision_clf = DecisionTreeClassifier(max_depth=10)
sgd_clf = SGDClassifier(max_iter=100, n_jobs=-1, loss='hinge')
log_clf = SGDClassifier(loss='log', max_iter=100, n_jobs=-1)
linear_svc = LinearSVC(loss='squared_hinge', dual=False, C=0.15, max_iter=100)
adaboost_clf = AdaBoostClassifier(base_estimator=random_clf, n_estimators=10)
knn_clf = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)

In [20]:
evaluate_model(linear_svc, vector_X, vector_Y, kfold=False)

0.5696428571428571

In [21]:
# voting classifier
baap_clf = VotingClassifier(estimators=[
                                        ('clf1', mnb_clf), 
                                        ('clf2', brnb_clf), 
                                        ('clf3', linear_svc), 
                                        ('clf4', sgd_clf)
                                        ],
                            voting='hard')


evaluate_model(baap_clf, vector_X, vector_Y, kfold=False)

0.577

##### Confusion Matrix

In [0]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [0]:
# confusion_ma/trix(evaluate_model(model, ))

### Create Submissions

In [0]:
predictions = list(label_encoder.inverse_transform(model.predict(vector_X_test)))

In [0]:
create_and_save_submission(predictions)

### Word2Vec Approach

In [0]:
from gensim.models import Word2Vec
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [0]:
stop_words_list = stopwords.words('english')
# stemmer = PorterStemmer()

pattern = re.compile(r'\b\w\w+\b')
processed_X = []
for sentence in X:
    processed_X.append([word.lower() if word.lower() not in stop_words_list else "stop_word" for word in re.findall(pattern, sentence.lower())])
    # processed_X.append([stemmer.stem(word) for word in re.findall(pattern, sentence.lower()) if word not in stop_words_list])
processed_X = np.asarray(processed_X)

### Train word2vec

In [25]:
w2v_model = Word2Vec(processed_X, size=100, window=5, min_count=1, workers=8, iter=50, sample=0)

2019-11-06 20:05:28,631 : INFO : collecting all words and their counts
2019-11-06 20:05:28,633 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-06 20:05:28,702 : INFO : PROGRESS: at sentence #10000, processed 403827 words, keeping 26591 word types
2019-11-06 20:05:28,781 : INFO : PROGRESS: at sentence #20000, processed 819912 words, keeping 38534 word types
2019-11-06 20:05:28,862 : INFO : PROGRESS: at sentence #30000, processed 1228962 words, keeping 47672 word types
2019-11-06 20:05:28,951 : INFO : PROGRESS: at sentence #40000, processed 1649577 words, keeping 55532 word types
2019-11-06 20:05:29,036 : INFO : PROGRESS: at sentence #50000, processed 2067376 words, keeping 62160 word types
2019-11-06 20:05:29,121 : INFO : PROGRESS: at sentence #60000, processed 2477292 words, keeping 68514 word types
2019-11-06 20:05:29,205 : INFO : collected 74121 word types from a corpus of 2884251 raw words and 70000 sentences
2019-11-06 20:05:29,206 : INFO : Loadin

### Test word2vec

##### Mean w2v Vectorizer

In [0]:
class MeanEmbeddingVectorizer():
    def __init__(self, word2vec):
        self.word2vec = word2vec.wv
        # if text is empty or no word found in the word2vec vocab - return vector of zeros
        self.dim = self.word2vec.vector_size

    def fit(self, X, y):
        return self

    def transform(self, X):
        # returns avg embedded vector for each X
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0) 
            for words in X
        ])


# def avg_feature_vector(words, model, num_features=100):
#     feature_vec = np.zeros((num_features, ), dtype='float32')
#     n_words = 0
#     vocab = model.wv.vocab
#     for word in words:
#         if word in vocab:
#             n_words += 1
#             feature_vec = np.add(feature_vec, model.wv[word])
#     if (n_words > 0):
#         feature_vec = np.divide(feature_vec, n_words)
#     return feature_vec

##### Tfidf w2v Vectorizer

In [0]:
class TfidfEmbeddingVectorizer():
    def __init__(self, word2vec):
        self.word2vec = word2vec.wv
        self.word2weight = None
        self.dim = self.word2vec.vector_size

    def fit(self, X, y):
        tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True, analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        # return weighted vecotr using tfidf
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)
                for words in X
            ])

### Blog
[link](https://github.com/nadbordrozd/blog_stuff/blob/master/classification_w2v/benchmarking_python3.ipynb)

[blog](http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/)

In [0]:
# start with naive bayes of the multinomial and bernoulli 
# with either pure counts or tfidf features
mult_nb = Pipeline([
                    ("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), 
                    ("multinomial nb", MultinomialNB(alpha=0.25))])
bern_nb = Pipeline([
                    ("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), 
                    ("bernoulli nb", BernoulliNB(alpha=0.25))])
mult_nb_tfidf = Pipeline([
                          ("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), 
                          ("multinomial nb", MultinomialNB(alpha=0.25))])
bern_nb_tfidf = Pipeline([
                          ("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), 
                          ("bernoulli nb", BernoulliNB(alpha=0.25))])
svc = Pipeline([
                ("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), 
                ("linear svc", SVC(kernel="linear"))])
svc_tfidf = Pipeline([
                      ("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), 
                      ("linear svc", SVC(kernel="linear", C=0.15))])

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

etree_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v_model)),
    ("extra trees", ExtraTreesClassifier(n_estimators=100))])

etree_w2v_tfidf = Pipeline([
    ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v_model)),
    ("extra trees", ExtraTreesClassifier(n_estimators=100))])

multi_nb_w2v_tfidf = Pipeline([
    ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v_model)),
    ("multinomial nb", MultinomialNB(alpha=0.25))])


In [173]:
all_models = [
    ("mult_nb", mult_nb),
    ("mult_nb_tfidf", mult_nb_tfidf),
    ("bern_nb", bern_nb),
    ("bern_nb_tfidf", bern_nb_tfidf),
    # ("svc", svc), # takes too long
    # ("svc_tfidf", svc_tfidf),
    ("w2v", etree_w2v),
    ("w2v_tfidf", etree_w2v_tfidf),
]

unsorted_scores = [(name, evaluate_model(model, processed_X, vector_Y, kfold=True)) for name, model in all_models]
scores = sorted(unsorted_scores, key=lambda x: -x[1])

from tabulate import tabulate
%matplotlib inline

print (tabulate(scores, floatfmt=".4f", headers=("model", 'score')))

Evaluating model performance using k-fold...
Evaluating model performance using k-fold...
Evaluating model performance using k-fold...
Evaluating model performance using k-fold...
Evaluating model performance using k-fold...
Evaluating model performance using k-fold...
model            score
-------------  -------
mult_nb_tfidf   0.5710
mult_nb         0.5571
bern_nb         0.5163
bern_nb_tfidf   0.5163
w2v_tfidf       0.3944
w2v             0.3608


In [0]:
w2v_X = []
for x in processed_X:
    w2v_X.append(avg_feature_vector(x, w2v_model))

w2v_X = np.asarray(w2v_X)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(w2v_X, vector_Y, test_size=0.2, random_state=42)

In [0]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model

In [0]:
# model = MultinomialNB()
model = AdaBoostClassifier(n_estimators=100, random_state=0)
# model = LinearSVC(random_state=0, tol=1e-5, C=1.0)
# print("Evaluation Accuracy: %0.5f"%evaluate_model(model, w2v_X, vector_Y))

model.fit(X_train, y_train)
print("Training Accuracy: %0.5f"%model.score(X_test, y_test))

Training Accuracy: 0.29864


In [0]:
# model = MultinomialNB()
model = DecisionTreeClassifier()
# model = LinearSVC(random_state=0, tol=1e-5, C=1.0)
# print("Evaluation Accuracy: %0.5f"%evaluate_model(model, w2v_X, vector_Y))

model.fit(X_train, y_train)
print("Accuracy: %0.5f"%model.score(X_test, y_test))

Accuracy: 0.19386


In [0]:
model = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
model.fit(X_train, y_train)
print("Accuracy: %0.5f"%model.score(X_test, y_test))

Accuracy: 0.34036
